# Three Methods of Prompt Configuration

This notebook demonstrates different approaches to configuring agent prompts, from simple strings to dynamic, context-aware prompts.

## Key Concepts
- **Method 1**: String Prompt (simplest)
- **Method 2**: SystemMessage (structured)
- **Method 3**: Callable/Dynamic Prompt (advanced)

## When to Use Each Method
- **String**: Simple, static agents
- **SystemMessage**: Production chat agents
- **Callable**: Adaptive, personalized agents

## Prerequisites

Make sure you have the required packages installed:

```bash
pip install --pre langchain langchain-community langchain-core langgraph pydantic
ollama pull qwen3
ollama serve
```

In [1]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
import tools

## Method 1: Simple String Prompt

The simplest approach - direct string instruction to the agent.

In [2]:
print("=== Method 1: Simple String Prompt ===")

# Create model instance
model = ChatOllama(model="qwen3")

# Method 1: String prompt (simplest)
agent1 = create_agent(
    model,
    [tools.helper_tool],
    prompt="You are a helpful assistant. Be concise and accurate in your responses."
)

print("✓ Agent 1 created with string prompt")
print("  Characteristics:")
print("  - Direct string instruction")
print("  - Simple and readable")
print("  - Good for basic use cases")
print("  - Easy to modify")

=== Method 1: Simple String Prompt ===
✓ Agent 1 created with string prompt
  Characteristics:
  - Direct string instruction
  - Simple and readable
  - Good for basic use cases
  - Easy to modify


## Method 2: SystemMessage Prompt

Using LangChain's SystemMessage class for more structured prompt handling.

In [3]:
print("=== Method 2: SystemMessage Prompt ===")

# Method 2: SystemMessage prompt (structured)
agent2 = create_agent(
    model,
    [tools.helper_tool],
    prompt=SystemMessage(
        content="You are a research assistant. Always cite your sources and provide detailed explanations."
    )
)

print("✓ Agent 2 created with SystemMessage prompt")
print("  Characteristics:")
print("  - Structured message object")
print("  - Better integration with chat models")
print("  - More explicit about message type")
print("  - Professional for production use")

=== Method 2: SystemMessage Prompt ===
✓ Agent 2 created with SystemMessage prompt
  Characteristics:
  - Structured message object
  - Better integration with chat models
  - More explicit about message type
  - Professional for production use


## Method 3: Dynamic Callable Prompt

A function that generates prompts based on state - the most flexible approach.

In [4]:
print("=== Method 3: Dynamic Callable Prompt ===")

# Method 3: Callable/Dynamic prompt (most flexible)
def dynamic_prompt(state):
    """Generate prompt based on user type and context."""
    user_type = state.get("user_type", "standard")
    
    if user_type == "expert":
        system_msg = SystemMessage(
            content="Provide detailed technical responses with code examples and advanced concepts."
        )
    elif user_type == "beginner":
        system_msg = SystemMessage(
            content="Provide simple, clear explanations suitable for beginners. Use analogies and avoid jargon."
        )
    else:  # standard
        system_msg = SystemMessage(
            content="Provide balanced explanations that are informative but accessible."
        )
    
    return [system_msg] + state["messages"]

agent3 = create_agent(model, [tools.helper_tool], prompt=dynamic_prompt)

print("✓ Agent 3 created with dynamic callable prompt")
print("  Characteristics:")
print("  - Adapts to user context")
print("  - Personalized responses")
print("  - Most flexible approach")
print("  - Perfect for adaptive systems")

=== Method 3: Dynamic Callable Prompt ===
✓ Agent 3 created with dynamic callable prompt
  Characteristics:
  - Adapts to user context
  - Personalized responses
  - Most flexible approach
  - Perfect for adaptive systems


## Testing All Three Agents

Let's test all agents with the same question to see their different behaviors:

In [5]:
# Test question for all agents
test_question = "Help me understand artificial intelligence"

print(f"Testing all agents with: '{test_question}'")
print("=" * 70)

# Test Agent 1 (String prompt)
print("\n=== Agent 1 Response (String Prompt) ===")
print("Expected: Concise and accurate response")
try:
    result1 = agent1.invoke({"messages": test_question})
    print(f"Response: {result1['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

# Test Agent 2 (SystemMessage prompt)
print("\n=== Agent 2 Response (SystemMessage Prompt) ===")
print("Expected: Detailed explanation with research focus")
try:
    result2 = agent2.invoke({"messages": test_question})
    print(f"Response: {result2['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

Testing all agents with: 'Help me understand artificial intelligence'

=== Agent 1 Response (String Prompt) ===
Expected: Concise and accurate response
Response: <think>
Okay, the user asked for help understanding artificial intelligence. I need to provide a clear and concise explanation. Let me start by defining AI as machines performing tasks that typically require human intelligence. Then, mention the different types: narrow AI (like virtual assistants) and general AI (which is theoretical). I should explain key areas like machine learning and natural language processing, and give examples. Also, include real-world applications such as healthcare and autonomous vehicles. Keep it structured but easy to follow, avoiding jargon. Make sure to ask if they need more details on any specific part. Let me check if I covered all the essentials without being too technical.
</think>

Artificial intelligence (AI) refers to machines or software designed to perform tasks that typically require hum

## Testing Dynamic Prompt with Different User Types

The dynamic prompt agent can adapt based on user context:

In [6]:
print("=== Agent 3 Responses (Dynamic Prompt) ===")

# Test with expert mode
print("\n--- Expert Mode ---")
print("Expected: Technical response with advanced concepts")
try:
    result3_expert = agent3.invoke({
        "messages": test_question,
        "user_type": "expert"
    })
    print(f"Response: {result3_expert['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

# Test with beginner mode
print("\n--- Beginner Mode ---")
print("Expected: Simple explanation with analogies")
try:
    result3_beginner = agent3.invoke({
        "messages": test_question,
        "user_type": "beginner"
    })
    print(f"Response: {result3_beginner['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

# Test with standard mode (default)
print("\n--- Standard Mode ---")
print("Expected: Balanced explanation")
try:
    result3_standard = agent3.invoke({
        "messages": test_question,
        "user_type": "standard"
    })
    print(f"Response: {result3_standard['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

print("\n💡 Notice how each agent responds differently based on its prompt configuration")

=== Agent 3 Responses (Dynamic Prompt) ===

--- Expert Mode ---
Expected: Technical response with advanced concepts
Response: <think>
Okay, the user wants to understand artificial intelligence. Let me start by breaking down what AI is. I should explain it in simple terms, maybe compare it to things they know. I need to cover the basics like definition, types of AI, common applications, and maybe touch on how it works without getting too technical. Also, it's important to mention both the benefits and some ethical considerations. Let me structure this so it's easy to follow. Wait, should I include examples? Yeah, examples like virtual assistants or recommendation systems could help. Oh, and maybe mention machine learning as a subset. I need to make sure the explanation is balanced, not too biased towards any one aspect. Let me check if there's anything else the user might need, like further resources or clarification on specific areas. Alright, time to put this all together in a clear, 